In [2]:
import numpy as np
import pandas as pd
from keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
datagen=ImageDataGenerator(
rescale=0.1/255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
fill_mode="nearest",
horizontal_flip=True,
validation_split=0.2)
df="/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train"
train_data=datagen.flow_from_directory(
df,
target_size=(256,256),
batch_size=32,
# class_mode=None,
subset="training")

val_data=datagen.flow_from_directory(
df,
target_size=(256,256),
batch_size=32,
# class_mode=None,
subset="validation")

Found 2903 images belonging to 2 classes.
Found 725 images belonging to 2 classes.


In [4]:
train_data

In [5]:
from keras.layers import Layer,BatchNormalization,Dropout,Conv2D,MaxPooling2D,Input,ReLU,LeakyReLU,Flatten,Dense
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.models import Model


# # Autoencoder

In [6]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1)

In [7]:
inputs=Input(shape=(256,256,3))
E=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)
E=MaxPooling2D(pool_size=(2,2),strides=(2,2))(E)

E=Conv2D(64,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=ReLU()(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)

E=MaxPooling2D(pool_size=(2,2),strides=(2,2))(E)
E=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)
E=MaxPooling2D(pool_size=(2,2),strides=(2,2))(E)

E=Conv2D(256,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=ReLU()(E)
E=Dropout(0.5)(E)
E=BatchNormalization()(E)
E=MaxPooling2D(pool_size=(2,2),strides=(2,2))(E)
E=Flatten()(E)
E=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=Dense(32,activation="relu")(E)
outputs=Dense(2,activation="sigmoid")(E)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     4,194,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            6

 Total params: 4,586,850 (17.50 MB)

 Trainable params: 4,585,890 (17.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [8]:
model.fit(train_data,epochs=5,batch_size=32,validation_data=val_data)

Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1730198539.767616     148 service.cc:145] XLA service 0x7c5e840146b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730198539.767671     148 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730198539.767676     148 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 1/91 ━━━━━━━━━━━━━━━━━━━━ 38:36 26s/step - accuracy: 0.3750 - loss: 11.9086

I0000 00:00:1730198555.699861     148 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


91/91 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.9653 - loss: 8.7472 - val_accuracy: 0.0000e+00 - val_loss: 144.3050
Epoch 2/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 63s 646ms/step - accuracy: 1.0000 - loss: 1.8539 - val_accuracy: 0.0000e+00 - val_loss: 89.7644
Epoch 3/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 64s 646ms/step - accuracy: 1.0000 - loss: 0.5264 - val_accuracy: 0.0000e+00 - val_loss: 34.4296
Epoch 4/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 67s 688ms/step - accuracy: 1.0000 - loss: 0.2074 - val_accuracy: 0.0000e+00 - val_loss: 10.5898
Epoch 5/5
91/91 ━━━━━━━━━━━━━━━━━━━━ 62s 640ms/step - accuracy: 0.9782 - loss: 2.5230 - val_accuracy: 1.0000 - val_loss: 0.8757


In [11]:
# !pip install cirq

In [12]:
# import cirq